<a href="https://colab.research.google.com/github/ranieri-unimi/faces-malchiodi-2022/blob/main/faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### run once

In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
import os
os.environ["KAGGLE_USERNAME"] = 'ranieriunimi'
os.environ["KAGGLE_KEY"] = str(hex(232307088475198570779809482024044346960))[2:]

In [ ]:
ref = 'bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows'
!mkdir datasets
!kaggle datasets download $ref --unzip -p ./datasets

In [ ]:
import nltk
nltk.download('all');

## ukraine

In [ ]:
SAMPLE_SIZE = 1024

In [ ]:
import numpy as np
import pandas as pd
import csv
import re
import string
import random

In [ ]:
import pyspark
import findspark

In [ ]:
# load dataset 
filename = r"./datasets/UkraineCombinedTweetsDeduped20220227-131611.csv.gzip"
pd.set_option("display.max_columns", None)
df = pd.read_csv(filename, compression='gzip', index_col=0, encoding='utf-8', quoting=csv.QUOTE_ALL)

## data cleaning

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

eng_sw = set(stopwords.words("english"))

In [ ]:
df = df[df.language == 'en'].text.tolist()

In [ ]:
if SAMPLE_SIZE:
    df = random.sample(df, SAMPLE_SIZE)

In [ ]:
# https://www.pluralsight.com/guides/building-a-twitter-sentiment-analysis-in-python

def preprocess_tweet_text(tweet):
    index, tweet = tweet

    tweet = tweet.lower()

    # cleanings 🧹

    # urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)

    # # @ and #
    # tweet = re.sub(r'\@\w+|\#','', tweet)

    # punctuations
    # tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet = tweet.translate(str.maketrans(string.punctuation+'…’”“', ' '*(len(string.punctuation)+4)))  # puntctuation to spaces


    # tweet_tokens = word_tokenize(tweet)

    # # emojitter
    # wrds = [e for word in tweet_tokens for e in re.findall(r"(\w+|[^\w ]+)", word)]
    # # TODO split also emoji-goups

    # # stopwords
    # filtered_words = [w for w in wrds if not w in eng_sw]
    
    # # stemmatize
    # ps = PorterStemmer()
    # stemmed_words = [ps.stem(w) for w in filtered_words]

    # # lemmatize
    # lemmatizer = WordNetLemmatizer()
    # lemma_words = [str(lemmatizer.lemmatize(w, pos='a')) for w in stemmed_words]
  
    # return (index, lemma_words)

    tweet = ' '.join(tweet.strip().split())

    return (index, tweet)

In [ ]:
#lang_hist = {l:df[df.language == l].size for l in df.language.unique()}

## hadoop instance

In [ ]:
# import findspark
# findspark.init("spark-3.1.1-bin-hadoop3.2") # SPARK_HOME
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
rdd = sc.parallelize(enumerate(df))

In [ ]:
rdd.take(5)

In [ ]:
dataPipe = rdd.map(preprocess_tweet_text)

In [ ]:
dataPipe.take(5)

In [ ]:
# indexing = dataset.flatMap(lambda _, v : [(e,e) for e in v]).reduceByKey(lambda k, v : k)

### foos

In [ ]:
def is_sub(sub, lst) : return all(e in lst for e in sub)
def add(a, b) : return a+b
def splat(t): return tuple(sorted(list(j for i in t for j in (i if isinstance(i, tuple) else (i,)))))
def doubled(t): return len(set(t)) == len(t)

In [ ]:
# nen_rdd = sc.parallelize(df)

In [ ]:
# elemListPipe = nen_rdd.flatMap(lambda word_list : word_list).distinct()
# elem_index = sc.parallelize(enumerate(elemListPipe.collect()))

### shingles

In [ ]:
shin_len = 4

In [ ]:
dataPipe.flatMap(lambda x : [(x[-1][i:i+shin_len], x[0]) for i in range(0, len(x[-1])-shin_len)]).distinct()

In [ ]:
import random
HSEED = 42
random.seed(HSEED)

In [ ]:
HBYTES = 2
HMAX = 2**(8*HBYTES)
SBOX = list(range(HMAX))
random.shuffle(SBOX)

In [ ]:
def phash(message):
  hash = len(message) % HMAX
  for i in message:
    hash = SBOX[(hash + ord(i)) % HMAX]
  return hash.to_bytes(HBYTES, 'big')